___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://cdn.brandfetch.io/idjblR01Uf/w/400/h/400/theme/dark/icon.jpeg?c=1dxbfHSJFAPEGdCLU4o5B" width="200px" height="180px" />

# <font color= #5aafc1> **PROYECTO FINAL** </font>
#### <font color= #2E9AFE> `Red Aeroportuaria Internacional`</font>
- <Strong> Sofía Maldonado, Viviana Toledo, César Adrián Santos </Strong>
- <Strong> Fecha </Strong>: 17/07/2025

___

In [ ]:
# Actualizar 
import importlib
import queries 
importlib.reload(queries)
from queries import DB_Queries

# Establecer conexión
db = DB_Queries('neo5j')

Añadir atributos de Continente a cada nodo:

In [8]:
query1 = """MATCH (a:Airport) 
WHERE a.country = 'Burma'
SET a.country = 'Myanmar'
"""

query2 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Kinshasa)'
SET a.country = 'Democratic Republic of the Congo'
"""

query3 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Brazzaville)'
SET a.country = 'Republic of the Congo'
"""

query4 = f"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ch0fas/MG_PROYECTO2/main/data/countries.csv' AS row
MATCH (a:Airport)
WHERE a.country = row.Country
SET a.continent = row.Continent
"""

db.custom_query(query4)

# <font color= #5aafc1> **Exploratory Data Analysis** </font>

#### <font color= #5aafc1> **4.2** </font>

Graph Schema:

Node Count:

In [ ]:
# 4.2.2
db.count_nodes('Airport')

,Airports
0,3214


Node Attributes:

In [10]:
db.node_attributes()

,nodeType,nodeLabels,propertyName,propertyTypes,mandatory
0,:`Airport`,[Airport],Id,[Long],True
1,:`Airport`,[Airport],latitude,[Double],True
2,:`Airport`,[Airport],country,[String],True
3,:`Airport`,[Airport],longitude,[Double],True
4,:`Airport`,[Airport],Label,[String],True
5,:`Airport`,[Airport],continent,[String],True


Relationship Count:

In [11]:
db.count_relationships('TO')

,TO_Count
0,36907


Relationship Attributes:

In [12]:
db.rel_attributes()

,relType,propertyName,propertyTypes,mandatory
0,:`TO`,distance,[Double],True


# <font color= #5aafc1> **Centrality & Community Algorithms** </font>

Subgraphs:

In [3]:
db.create_subgraph('Airport', 'TO', 'NATURAL','distance')

Successfully created Directed Subgraph with Airport nodes and TO relationships!


In [4]:
db.create_subgraph('Airport', 'TO', 'UNDIRECTED','distance')

Successfully created Undirected Subgraph with Airport nodes and TO relationships!


Centrality Algorithms:

In [6]:
db.degrees('Airport')
db.page_rank('Airport')
db.betwenness('Airport')
db.closeness('Airport')

Degree Attribute added Successfully!
In_Degree Attribute added Successfully!
Page Rank Attribute added Successfully!
Betweenness Attribute added Successfully!
Closeness Attribute added Successfully!


# <font color= #5aafc1> **Path Finding Algorithms** </font>

#### <font color= #5aafc1> **4.3** </font>

In [4]:
# Le voy a agregar atributos de latitud y longitud en integer para poder meterlo a Gephi 

query4_3 = f"""
    MATCH (n)
    SET n.latInt = toInteger(n.latitude)
    SET n.longInt = toInteger(n.longitude)
"""

db.custom_query(query4_3)

""


#### <font color= #5aafc1> **4.5** </font>

In [35]:
query4_5 = f""" 
    MATCH(a:Airport {{Label:'Villahermosa'}})-[r1]->(b)-[r2]->(c)-[r3]->(m:Airport {{Label:'Delhi'}})
    WHERE r3.distance IS NOT NULL
    RETURN a.Label,r1.distance AS Dist1,b.Label,r2.distance AS Dist2,c.Label,r3.distance AS Dist3,m.Label
    ORDER BY Dist1+Dist2+Dist3 DESC
    LIMIT 10;
"""

db.custom_query(query4_5)

,a.Label,Dist1,b.Label,Dist2,c.Label,Dist3,m.Label
0,Villahermosa,1358.755582,Houston,10681.305092,Tokyo,5915.459645,Delhi
1,Villahermosa,1358.755582,Houston,11358.558584,Seoul,4657.687448,Delhi
2,Villahermosa,1358.755582,Houston,12947.031367,Doha,2552.069255,Delhi
3,Villahermosa,1358.755582,Houston,11569.256010,Beijing,3813.207294,Delhi
4,Villahermosa,1358.755582,Houston,13139.523542,Dubai,2186.045810,Delhi
5,Villahermosa,678.579085,Mexico City,9209.229377,Paris,6572.730383,Delhi
6,Villahermosa,678.579085,Mexico City,9855.892994,Munich,5906.062233,Delhi
7,Villahermosa,678.579085,Mexico City,9562.676617,Frankfurt,6127.480196,Delhi
8,Villahermosa,678.579085,Mexico City,8913.836258,London,6738.831316,Delhi
9,Villahermosa,678.579085,Mexico City,9216.602463,Amsterdam,6370.585848,Delhi


#### <font color= #5aafc1> **4.8. Imagina que vives en el país que elegiste en el punto 4.7 (Colombia) y tienes la posibilidad de viajar a cualquier parte del mundo desde su aeropuerto más importante.** </font>

Primero encontraremos el aeropuerto más importante de Colombia en base a la medida de centralidad _Page Rank_, que evalúa las conexiones que salen del nodo y les asigna una relevancia.

In [14]:
query1 = """MATCH (a:Airport)
WHERE a.country = 'Colombia'
RETURN a.Label AS Airport, a.country AS Country, a.Page_Rank AS Page_Rank
ORDER BY Page_Rank DESC
LIMIT 1
"""

db.custom_query(query1)

,Airport,Country,Page_Rank
0,Bogota,Colombia,9.420864


El aeropuerto más importante de Colombia es el de Bogotá, que se ubica en la capital.

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.1. ¿Cuántos kilómetros se requieren para visitar las 7 maravillas del mundo?** </font>

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.2. ¿Cuál sería la mejor ruta para visitarlas todas, en qué orden y por qué?** </font>

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.3. Aeropuertos visitables desde el Aeropuerto más importante de Colombia en menos de 20,000km** </font>

Crearemos un atributo que nos permita evaluar todas las rutas posibles desde Bogotá:

In [ ]:
db.delta_pathing('Airport', 'Label', 'Bogota', 'distance')

Los aeropuertos y rutas visitables que sumen menos de 20,000 km:

In [ ]:
query2 = """MATCH (source:Airport) -[r:PATH_Bogota_Directed]-> (target:Airport)
WHERE r.totalCost <= 20_000
RETURN source.Label AS Source, target.Label AS Target, r.totalCost AS Kilometers
"""

db.custom_query(query2)